In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10), place idx에서 23개가 비어있음
product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5834,11), product idx에서 538개가 비어있음
video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9), video idx에서 315개가 비어있음

In [12]:
index_null_del_user_place = pd.read_csv('../Data/whyout_data/index_null_del_user_place.csv', index_col=0).astype(dtype='float16')
index_null_del_user_product = pd.read_csv('../Data/whyout_data/index_null_del_user_product.csv', index_col=0).astype(dtype='float16')
index_null_del_user_video = pd.read_csv('../Data/whyout_data/index_null_del_user_video.csv', index_col=0).astype(dtype='float16')

In [13]:
cosine_sim = cosine_similarity(index_null_del_user_place)
cosine_sim_df = pd.DataFrame(cosine_sim)

In [137]:
def item_recommendations(target_user, matrix, user_item_matrix, place, k=10):
    # 가장 유사한 k명의 유저 인덱스 찾기
    recom_idx = matrix[target_user].values.argsort()[::-1][1:k+1]
    
    # 유효한 유저 인덱스 필터링
    valid_recom_idx = [idx for idx in recom_idx if idx >= 0 and idx < len(user_item_matrix)]
    valid_recom_idx = valid_recom_idx[:k]
    
    # 추천 아이템 리스트 초기화
    recommend_list = []
    for user_id in valid_recom_idx:
        # 해당 유저가 user_item_matrix에서 좋아했던 아이템들 (아이템의 값이 1 이상인 경우)
        liked_items = user_item_matrix.iloc[user_id,:]
        liked_items_idx = liked_items[liked_items > 0].index.tolist()
        # 추천 리스트에 아이템 추가 (중복 제거)
        recommend_list.extend([item for item in liked_items_idx if item not in recommend_list])
        
    # 상위 k개의 추천 아이템 선택
    recommend_list = list(map(int, recommend_list[:k]))
    place_idx = place['idx'].astype(int).tolist()
    valid_recommend_list = [idx for idx in recommend_list if idx in place_idx]

    # 추천 아이템들의 값을 DataFrame 형태로 반환
    recommended_items_info = place[place['idx'].isin(valid_recommend_list)]
    
    return pd.DataFrame(recommended_items_info)

In [138]:
item_recommendations(1, cosine_sim_df, index_null_del_user_place, place)

,idx,지역(위치정보),공간유형,주변환경,계절,테마,클릭 수,좋아요 수,북마크 수,댓글 수
7,8,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[0,0,0,1,0,0,0,0]","[1,0,0,0]","[1,1,1,1,1,1,0,0,0,1,0,0,0,1,0]",0.333349,0.184685,0.253769,0.000000
1169,1170,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[0,1,0,1,0,0,0,0]","[1,0,0,0]","[1,1,1,1,1,1,0,0,0,1,0,1,0,0,0]",0.169633,0.081081,0.109296,0.000000
1364,1365,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[0,0,0,1,0,0,0,0]",NaN,"[1,1,1,1,1,1,0,0,0,1,0,0,0,0,0]",0.164326,0.031532,0.089196,0.000000
1402,1403,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[0,0,0,1,0,0,0,0]",NaN,"[1,1,1,1,0,1,1,0,0,1,0,0,0,0,1]",0.207956,0.171171,0.192211,0.000000
1506,1507,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[0,0,0,1,0,1,0,0]",NaN,"[1,1,1,1,1,1,0,0,1,0,0,0,0,0,0]",0.267036,0.346847,0.368090,0.000000
1807,1808,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[1,0,0,0,0,0,0,0]",NaN,"[1,1,1,1,1,0,0,0,0,1,0,1,0,1,0]",0.256375,0.211712,0.214824,0.000000
2690,2691,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[0,1,1,1,0,0,0,0]",NaN,"[1,1,1,1,1,0,1,0,1,1,0,0,0,1,0]",0.311135,0.225225,0.257538,0.000000
2990,2991,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[0,1,0,1,0,1,0,0]",NaN,"[1,1,0,1,1,0,1,0,0,0,0,0,0,0,0]",0.246513,0.220721,0.227387,0.000000
3404,3406,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0,1,0,0,0,0,0,0,0]","[1,0,0,0,0,0,0,0]",NaN,"[0,0,0,0,0,0,0,0,0,0,0,1,0,1,0]",0.260602,0.148649,0.206030,0.222222
3694,3697,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[0,0,0,0,0,1,0,0,0]","[0,0,0,0,0,0,0,1]","[1,0,0,0]","[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]",0.002160,0.004505,0.003769,0.000000
